In [1]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

### Chicago
Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

There is also a CSV at https://data.cityofchicago.org/Health-Human-Services/Daily-COVID-19-Cases-and-Deaths/kxzd-kd6a, but this appears to be based on reported dates for deaths, which is not clean.

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [2]:
chi_death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 2,
                                   22: 2, 23: 2, 24: 2, 25: 5, 26: 2,
                                   27: 7, 28: 4, 29: 21, 30: 12,
                                   31: 16, 32: 24, 33: 14, 34: 24,
                                   35: 21, 36: 26, 37: 22, 38: 25,
                                   39: 34, 40: 26, 41: 30, 42: 33,
                                   43: 32, 44: 31, 45: 33, 46: 32,
                                   47: 30, 48: 38, 49: 36, 50: 34,
                                   51: 35, 52: 32, 53: 45, 54: 33,
                                   55: 49, 56: 37, 57: 46, 58: 31,
                                   59: 45, 60: 39, 61: 33, 62: 33,
                                   63: 46, 64: 40, 65: 45, 66: 52,
                                   67: 38, 68: 38, 69: 29, 70: 34,
                                   71: 21, 72: 8})

In [3]:
chi_last_day = max(chi_death_dict.keys())

In [4]:
chi_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + chi_last_day)]
chi_deaths =[chi_death_dict[day - 28] for day in range(29 + chi_last_day)]

In [5]:
chi_death_df = pd.DataFrame(chi_deaths, index=chi_index, columns=['deaths'])

In [6]:
# Inspect that last few days
chi_death_df.tail()

,deaths
2020-05-07,38
2020-05-08,29
2020-05-09,34
2020-05-10,21
2020-05-11,8


In [7]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')